In [1]:
import festim as F
import h_transport_materials as htm
import numpy as np

[DESKTOP-6T0UN3L:65410] mca_base_component_repository_open: unable to open mca_btl_openib: librdmacm.so.1: cannot open shared object file: No such file or directory (ignored)


In [4]:
# eurofer properties
diffusivity_eurofer = htm.diffusivities.filter(material="eurofer_97").filter(
    author="chen"
)
D_eurofer = diffusivity_eurofer[0]

solubility_eurofer = htm.solubilities.filter(material="eurofer_97").filter(
    author="chen"
)
S_eurofer = solubility_eurofer[0]

results_folder = "challenge_B/"

my_model = F.Simulation(log_level=40)

# define mesh
size = 5e-03
vertices = np.linspace(0, size, num=1000)

my_model.mesh = F.MeshFromVertices(vertices=vertices, type="cylindrical")

# define materials
eurofer = F.Material(
    id=1,
    borders=[0, size],
    D_0=D_eurofer.pre_exp.magnitude,
    E_D=D_eurofer.act_energy.magnitude,
)
my_model.materials = F.Materials([eurofer])

# define temperature
my_model.T = F.Temperature(value=600)

# define boundary conditions
my_model.boundary_conditions = [
    F.SievertsBC(
        surfaces=1,
        S_0=S_eurofer.pre_exp.magnitude,
        E_S=S_eurofer.act_energy.magnitude,
        pressure=1e6,
    ),
    F.DirichletBC(surfaces=2, value=0, field="solute"),
]

# define settings
my_model.settings = F.Settings(
    absolute_tolerance=1e-10,
    relative_tolerance=1e-10,
    maximum_iterations=30,
    transient=False,
)

# define exports
my_derived_quantities = F.DerivedQuantities(
    [F.SurfaceFlux("solute", surface=2)],
    filename=results_folder + "derived_quantities.csv",
)
my_model.exports = F.Exports([my_derived_quantities])

# run simulation
my_model.initialise()
my_model.run()

Defining initial values
Defining variational problem
Defining source terms
Defining boundary conditions
Solving steady state problem...
Solved problem in 0.00 s
Surface flux = 1.50e+02 H/m2/s


/home/jdark/anaconda3/envs/festim-env/lib/python3.11/site-packages/festim/exports/derived_quantities/derived_quantities.py:52: DeprecationWarning: The derived_quantities attribute will be deprecated in a future release, please use festim.DerivedQuantities as a list instead
  warnings.warn(


In [5]:
data = np.genfromtxt(
    results_folder + "derived_quantities.csv", delimiter=",", names=True
)

surface_flux = data["Flux_surface_2_solute"] * -1

print(f"Surface flux = {surface_flux:.2e} H/m2/s")

Surface flux = 1.50e+02 H/m2/s
